## 笔试题目

当时笔试做的时候没有做完，核心思路找到了但是提出的更新公式有点问题，主要在于只加了Distance_2而忘了减去Map(i1,j1)。具体题目见：[Link](https://blog.csdn.net/u012465304/article/details/81180707)。盲写写完，fix了2个小bug，一次跑出正确答案，开心得一批！

<pre>
<b>
输入：
N = 3
M = 2

Map = {
 {0, 2, 3},
 {2, 0, 1},
 {3, 1, 0}
}

输出：
result = {
 {4, 4, 3},
 {4, 2, 5},
 {3, 5, 2}
}
</b>
</pre>



更新公式：
$$Distance_M(i,j)=Distance_{M-1}(i,j)+\min_{i1,j1}[(Distance_{2}(i1,j1)) - Map(i1,j1)], (i1,j1)是M-1路径上的任意连续两点$$ 

先求$Distance_{2}(i,j)$； <br>
1. 若`i==j`，则为找到到i最近的点的距离乘2；<br>
1. 若`i!=j`，则需要找到$\min(Map(i,k)+Map(k,j))$




In [22]:
N = 3
M = 2
Map = [
 [0, 2, 3],
 [2, 0, 1],
 [3, 1, 0]
    ]
def Solve(N,M,Map):
    dis2_results = [[] for i in range(N)] 
    dis2_path = [[] for i in range(N)] 
    

    # 求Distance_2
    for i in range(N):
        for j in range(i+1): # 由对称性，第0行，只遍历0,0；第二行,0，1和1，1；。。。
            min_value = 99999999
            min_index = None
            if i==j:
                min_value = min([v for v in Map[i] if v >0]) # 中间节点可以是除了i,j外的任意点；
                min_index = Map[i].index(min_value)
                min_value *= 2  # 乘以2是因为需要一来一回, 但注意需要在找到index以后再乘
                
            else:
                for k in range(N):
                    if k!=i and k!=j:
                        d = Map[i][k] + Map[k][j]
                        if d<min_value:
                            min_value = d
                            min_index = k
                        else:continue    
                    else:continue           
            dis2_results[i].append(min_value)
            dis2_path[i].append(min_index) #
        
    results = dis2_results  # M step distance
    tmp_path = dis2_path  # M-1 step path
    count=2 # start from 2; if M==2, break directly
    while count<M:
        for i in range(N):
            for j in range(i+1): # i>=j
                min_value = 99999999
                min_index = None
                for index,(i1, j1) in enumerate(zip(tmp_path[i][j][:-1], tmp_path[i][j][1:])): # 连续的前后两个
                    d = dis2_path[i1][j1] - Map[i1][j1]
                    if d<min_value:
                        min_value = d
                        min_index = [index, i1,j1] 
                    else:continue
                results[i][j] = results[i][j] + min_value
                tmp_path[i][j] = tmp_path[i][j][:index+1]+ dis2_path[i][j] + tmp_path[i][j][index+1:]
    
    # 注意，需要根据对角线性质补齐矩阵
    for i in range(N):
        for j in range(i+1,N): # j>i Here
            results[i].append(results[j][i])
    
    return results
                
                
%timeit -n 1 Solve(N,M,Map)    

1 loop, best of 3: 13.6 µs per loop


In [12]:
def solve(str):
    if len(str)<=1:
        return(str)
    else:
        start_index = -1
        end_index = -1
        max_length = 1
         
        length = len(str)

        for i in range(length):
            current_substr = set() # save all char in the current substr
            current_substr.update(str[i]) 
            current_length = 1
            current_start_index= i
            for j in range(i, length):
                if str[j] not in current_substr:
                    current_substr.update(str[j])
                    current_length+=1
                    if j == length-1:
                        current_end_index = j
                        if current_length>max_length:
                            max_length = current_length
                            start_index = current_start_index
                            end_index = current_end_index
                        else:break  
                else:
                    current_end_index = j-1
                    
                    if current_length>max_length:
                        max_length = current_length
                        start_index = current_start_index
                        end_index = current_end_index
                    else:break    
        
        return max_length, start_index, end_index

print(solve('abcabcbb'))   

(1, -1, -1)


In [52]:

# 连续； 不含重复字符； 边界

def solve(str):

    '''
        str <String>: 指定字符串
        
        Return:
            max_length <Int>: 最大的不含重复字符的最长子串的长度 
    '''
    
    if len(str)<=1:
        return(str)
    else:
        start_index = -1
        end_index = -1
        max_length = 1
         
        length = len(str)
        
        # 两层循环
#        for i in range(length):
#            current_substr = set() # save all char in the current substr
#            current_substr.update(str[i]) 
#            current_length = 1
#            current_start_index= i
#            for j in range(i+1, length): 
#                if str[j] not in current_substr:
#                    current_substr.update(str[j])
#                    current_length+=1
#                    if j == length-1: # 考虑边界
#                        current_end_index = j
#                        if current_length>max_length:
#                            max_length = current_length
#                            start_index = current_start_index
#                            end_index = current_end_index
#                        else:pass    
#                        break # 跳出j层循环
#                else:
#                    current_end_index = j-1
#                    if current_length>max_length:
#                        max_length = current_length
#                        start_index = current_start_index
#                        end_index = current_end_index
#                    else:pass    
#                    break # 跳出j层循环
        # 一层循环
        # 遇到重复的，比较最大，从前面重复的部分截断，并减去相应长度变化; 则重复则持续加1；
        current_substr = set() # save all char in the current substr
        start_index = 0
        current_length = 0
        for i in range(length):
            if str[i] in current_substr:
                index = str.index(str[i])
                str = str[:index] + '_' + str[index+1:] # 有除了'_'别的更好的替换
                if current_length>max_length:
                    max_length = current_length
                current_length = current_length - (index+1-start_index) + 1
                for j in range(start_index, index):
                    current_substr.remove(str[j])
                start_index = index+1
            else:
                current_substr.update(str[i]) 
                current_length+=1
                if i==length-1:
                    if current_length>max_length:
                        max_length = current_length
                    else:pass    
        
        return max_length

# TestCase
for str in ['aaaa','abcabcbb', 'pwwkew']:
    print(solve(str))
            


1
3
3


In [25]:
'a' not in set(['a'])

False